In [6]:
from BackTesting_System.backtesting import EconomicIndicators,BackTesting

In [7]:
import pandas as pd

In [10]:
ec = EconomicIndicators()
bt = BackTesting()

In [14]:
df = pd.read_csv('C:/Users/smleo/Desktop/Ai/mom.csv',index_col=0)

In [15]:
df.head(1)

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,20_mom
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트,NaN,NaN,NaN


In [16]:
def make_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        con1=(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)<100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)>=95)
        con2=(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)>=100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)<110)
        con3=(df.loc[idx[0]:idx[-1],'5_mom']>=df.loc[idx[0]:idx[-1],'5_mom'].shift(1))&(df.loc[idx[0]:idx[-1],'5_mom']<110)
        df.loc[idx[0]:idx[-1], 'signal'] = con1&con2&con3
    return df

In [80]:
def make_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        '''buy 조건 '''
        con1=(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)<100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)>=95)
        con2=(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)>=100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)<110)
        con3=(df.loc[idx[0]:idx[-1],'5_mom']>=df.loc[idx[0]:idx[-1],'5_mom'].shift(1))&(df.loc[idx[0]:idx[-1],'5_mom']<110)
        con4=(df.loc[idx[0]:idx[-1],'누적거래대금']>=10000000000)
        df.loc[idx[0]:idx[-1], 'buy'] = con1&con2&con3&con4
        '''sell 조건 '''
        con5=(df.loc[idx[0]:idx[-1],'buy'].shift(5)==True) #5일뒤 매도하는 조건
        df.loc[idx[0]:idx[-1], 'sell'] = con5
    return df

In [19]:
def search_vol(df,date,range):
    df = df[(df['누적거래대금']>=range)&(df['날짜']==date)]
    return df

In [20]:
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [113]:
df_test = make_signal(df)

In [143]:
codes = codes.rename(columns={"회사명":"종목명"})

In [132]:
codes = pd.read_csv('SRC\current_kospi_codes.csv')

In [144]:
inner_join_result = df_test.merge(codes, on='종목명', how='inner')
inner_join_result


,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,20_mom,buy,sell,구매수량,종목코드,상장일
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882122,2022-12-23,3795,3885,3930,3755,50452,191959725,한진중공업홀딩스,96.687898,95.591940,90.572792,False,False,3,3480,1956-03-03
882123,2022-12-26,3785,3795,3800,3650,75667,282358160,한진중공업홀딩스,96.188056,94.862155,95.100503,False,False,3,3480,1956-03-03
882124,2022-12-27,3810,3750,3840,3750,30791,116935765,한진중공업홀딩스,97.442455,95.250000,93.268054,False,False,3,3480,1956-03-03
882125,2022-12-28,3890,3800,3930,3770,14512,55566740,한진중공업홀딩스,101.302083,96.526055,92.840095,False,False,3,3480,1956-03-03


In [145]:
df_test = inner_join_result

In [147]:
df_test=df_test.rename(columns={'종목코드':'코드'})

In [187]:
bt.MONEY=10000000

In [188]:
bt.buy(df=df_test,target='buy')
bt.MONEY

7673959

In [191]:
df_test[df_test['buy']==True]

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,20_mom,buy,sell,구매수량,코드,상장일
112,2018-05-31,38900,39400,39650,37400,303222,11703371900,LX세미콘,108.659218,113.411079,103.595206,True,False,3,108320,2022-11-03
369,2019-06-20,44400,42700,46000,42600,401686,17976324250,LX세미콘,107.117008,106.474820,108.029197,True,False,3,108320,2022-11-03
470,2019-11-15,37150,36000,37550,35450,286537,10519224650,LX세미콘,107.215007,111.897590,110.565476,True,False,3,108320,2022-11-03
624,2020-07-01,40550,39500,41550,39250,448430,18229491050,LX세미콘,105.461638,106.710526,100.996264,True,False,3,108320,2022-11-03
644,2020-07-29,49600,48150,49800,47400,274033,13404255550,LX세미콘,108.891328,106.209850,122.318126,True,False,3,108320,2022-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878232,2021-12-17,32750,32400,33100,31550,375098,12304841750,메리츠화재,106.158833,100.152905,112.736661,True,False,3,60,1956-07-02
878242,2022-01-03,34500,33300,35050,33300,302553,10413548750,메리츠화재,105.990783,105.343511,105.504587,True,False,3,60,1956-07-02
878288,2022-03-15,39200,39200,40250,38350,440953,17510537150,메리츠화재,105.945946,91.162791,88.487585,True,False,3,60,1956-07-02
878308,2022-04-12,48600,48100,49200,47700,226876,11012616950,메리츠화재,104.068522,107.284768,123.979592,True,False,3,60,1956-07-02


In [189]:
bt.ASSET

{89590: 3,
 14680: 3,
 8700: 3,
 660: 3,
 3060: 3,
 14160: 3,
 207940: 3,
 86790: 3,
 34220: 3,
 24110: 3,
 990: 3,
 5930: 3}

In [186]:
bt.sell(df=df_test, target='sell')
bt.MONEY

KeyError: 251270

#### Raw File 이 없을 당신을 위해 

In [193]:
display(df_test.head(50))
display(df_test.info())

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,20_mom,buy,sell,구매수량,코드,상장일
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트,NaN,NaN,NaN,False,False,3,377740,2022-12-22
5,2022-12-29,8580,8890,8920,8450,2059794,17721767120,바이오노트,80.563380,NaN,NaN,False,False,3,377740,2022-12-22
6,2022-11-28,7420,10200,10450,7420,3458712,27586626590,유니드비티플러스,NaN,NaN,NaN,False,False,3,446070,2022-11-28
7,2022-11-29,6900,7330,7350,6860,834344,5849468050,유니드비티플러스,NaN,NaN,NaN,False,False,3,446070,2022-11-28
8,2022-11-30,6360,6960,7010,6340,855296,5697738920,유니드비티플러스,NaN,NaN,NaN,False,False,3,446070,2022-11-28
9,2022-12-01,6330,6470,6530,6320,431925,2764739060,유니드비티플러스,NaN,NaN,NaN,False,False,3,446070,2022-11-28


<class 'pandas.core.frame.DataFrame'>
Int64Index: 882127 entries, 0 to 882126
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   날짜      882127 non-null  object 
 1   종가      882127 non-null  int64  
 2   시가      882127 non-null  int64  
 3   최고가     882127 non-null  int64  
 4   최저가     882127 non-null  int64  
 5   누적거래량   882127 non-null  int64  
 6   누적거래대금  882127 non-null  int64  
 7   종목명     882127 non-null  object 
 8   5_mom   878312 non-null  float64
 9   10_mom  874501 non-null  float64
 10  20_mom  866881 non-null  float64
 11  buy     882127 non-null  object 
 12  sell    882127 non-null  object 
 13  구매수량    882127 non-null  int64  
 14  코드      882127 non-null  int64  
 15  상장일     882127 non-null  object 
dtypes: float64(3), int64(8), object(5)
memory usage: 114.4+ MB


None